In [ ]:
from math import e,sqrt

layer1 = arcpy.GetParameterAsText(0)
layer2 = arcpy.GetParameterAsText(1)
field = arcpy.GetParameterAsText(2)
bata = arcpy.GetParameterAsText(3)

fieldNameList = ['gravity','maximum','assigned']
arcpy.DeleteField_management(layer1,fieldNameList)
arcpy.AddField_management(layer1,'gravity','DOUBLE',12,6)
arcpy.AddField_management(layer1,'maximum','DOUBLE',12,6)
arcpy.AddField_management(layer1,'assigned','SHORT',6)

result = arcpy.GetCount_management(layer1)
records = int(result.getOutput(0))

i = 0

places = arcpy.UpdateCursor(layer1)
for place in places:
    i+=1
    arcpy.SetProgressorLabel(str(i)+'out of'+str(records))
    
    query = place.shape.centroid
    total = 0.0
    max_value = 0.0
    max_id = 0
    
    facilities = arcpy.SearchCursor(layer2)
    for facility in facilities:
        d = facility.shape.distanceTo(query)
        accessibility = pow(e, d * float(bata))
        attractiveness = facility.getValue(field)
        gravity = attractiveness / accessibility
        total = total + gravity
        
        if gravity > max_value:
            max_value = gravity
            max_id = facility.OBJECTID
    
    place.gravity = total
    place.maximum = max_value
    place.assigned = max_id
    
    places.updateRow(place)
            
    
    
